# Missing VMs (BMC Discovery)

This notebook reproduces the DisMAL `missing_vms` report: Hypervisor hosts that have VMs which have not been discovered (no container host).
It reads connection details from `config.yaml`, queries the appliance, and writes `missing_vms.csv` under `output_<target>`.

## Requirements

We use `tideway` from pip (remote), plus `pandas` and `PyYAML`.
Uncomment the following to install in your environment if needed.

In [ ]:
# %pip install -q requests pandas pyyaml

import pandas as pd
import requests
import yaml
from pathlib import Path
from urllib.parse import urljoin
import ipaddress
import json, os
import math
import tideway
import re

## Configuration (from config.yaml)

Reads settings from `../config.yaml` including target, token/token_file,
API version, and SSL verification preference.

In [ ]:
def load_config_params(
    start: Path,
    appliance_name: str = None,
    appliance_index: int = 0,
) -> dict:
    def _find_repo_root(start: Path) -> Path:
        for p in [start] + list(start.parents):
            if (p / 'config.yaml').exists():
                return p
        return start.parent

    repo_root = _find_repo_root(start)
    config_path = repo_root / 'config.yaml'

    with open(config_path, 'r') as fh:
        cfg = yaml.safe_load(fh) or {}

    apps = cfg.get('appliances') or []
    selected = None
    if isinstance(apps, list) and apps:
        if appliance_name:
            selected = next((a for a in apps if a.get('name') == appliance_name), None)
            if selected is None:
                raise ValueError(f"No appliance named '{appliance_name}' in config.yaml")
        else:
            try:
                selected = apps[int(appliance_index)]
            except Exception:
                selected = apps[0]

    target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
    if not target:
        raise ValueError('config.yaml missing "target"')

    token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
    token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
    if not token and token_file:
        tf_path = Path(token_file)
        if not tf_path.is_absolute():
            tf_path = repo_root / tf_path
        with open(tf_path, 'r') as tf:
            token = tf.read().strip()
    if not token:
        raise ValueError('API token not found in config.yaml (token or token_file)')

    api_version = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
    verify_ssl = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

    sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
    output_dir = repo_root / f'output_{sanitized}'
    output_dir.mkdir(parents=True, exist_ok=True)

    return {
        "repo_root": repo_root,
        "config_path": config_path,
        "cfg": cfg,
        "selected": selected,
        "target": target,
        "token": token,
        "api_version": api_version,
        "verify_ssl": verify_ssl,
        "output_dir": output_dir,
    }

In [ ]:
def init_appliance(appliance_name: str = "prod"):
    params = load_config_params(Path.cwd(), appliance_name=appliance_name)

    target = params["target"]
    api_version = params["api_version"]
    verify_ssl = params["verify_ssl"]
    output_dir = params["output_dir"]

    print('Base Host     :', target)
    print('API Version   :', api_version)
    print('Verify SSL    :', verify_ssl)
    print('Output folder :', output_dir)

    api_number = api_version.lstrip('v')
    app = tideway.appliance(target, params["token"], api_version=api_number, ssl_verify=verify_ssl)

    try:
        about = app.api_about
        print('Appliance reachable:', about.status_code)
    except Exception as e:
        print('Warning: failed to contact appliance /api/about:', e)

    return {
        "params": params,
        "target": target,
        "app": app,
        "api_version":api_number,
        "output_dir":output_dir,
    }

# Initialise Instances

In [ ]:
print("Initialise Prod:")
twprod = init_appliance("prod")

print("Initialise Dev:")
twdev = init_appliance("dev")

In [ ]:
# bootstrapping variables

token = twprod["params"]["token"]
tokendev = twdev["params"]["token"]
VERIFY_SSL = twprod["params"]["verify_ssl"]
verify_ssl_dev = twdev["params"]["verify_ssl"]
target = twprod["target"]
target_dev = twdev["target"]
api_version = twprod["api_version"]
api_version_dev = twdev["api_version"]
output_dir = twprod["output_dir"]
output_dir_dev = twdev["output_dir"]

BASE_URL = target if ('://' in target) else f'https://{target}'
API_VERSION = api_version

## Run search and normalize

Executes the query via Tideway Data API and converts to a DataFrame.

In [ ]:
def get_results(instance, qry, columns=['No Results']):
    results = instance['app'].data().search({'query': qry}, format='object', limit=500)
    df = pd.DataFrame(results) if results else pd.DataFrame()
    if df.empty:
        # Provide headers if no rows returned
        df = pd.DataFrame(columns=columns)
    if 'Discovery Instance' not in df.columns:
        df.insert(0, 'Discovery Instance', instance['target'])
    else:
        df['Discovery Instance'] = instance['target']
    return df

def run_and_display(instance, query, columns, head: int = 5):
    """
    Run get_results for an instance, print the target, and display the head of the DataFrame.
    """
    df = get_results(instance, query, columns)
    print(instance['target'])
    display(df.head(head))
    return df

# Fetch tables
qry = '''
                    search VirtualMachine
                    where nodecount(traverse HostContainer:HostContainment:ContainedHost:) = 0
                    show
                    vm_type as 'VirtualMachine.vm_type',
                    (product_version or cloud_class) as 'VirtualMachine.product_version',
                    #RunningSoftware:HostedSoftware:Host:.name as 'Host.name',
                    #RunningSoftware:HostedSoftware:Host:.type as 'Host.type',
                    vm_name as 'VirtualMachine.vm_name',
                    vm_guest_os as 'VirtualMachine.vm_guest_os',
                    guest_full_name as 'VirtualMachine.guest_full_name',
                    (vm_status or cloud and "Cloud Hosted") as 'VirtualMachine.vm_status'
                '''

cols= [
    "VirtualMachine.vm_type","VirtualMachine.product_version","Host.name","VirtualMachine.vm_name",
    "VirtualMachine.vm_guest_os","VirtualMachine.guest_full_name","VirtualMachine.vm_status"
      ]

_prod = run_and_display(twprod, qry, cols)
_dev = run_and_display(twdev, qry, cols)

## Save CSV

Insert 'Discovery Instance' as the first column and save to `output_<target>/missing_vms.csv`.

In [ ]:
def save(df: pd.DataFrame, output_dir: Path, filename: str):
    """
    Save a discovery run DataFrame to CSV in the specified output directory.
    """
    output_csv = str(output_dir / f"{filename}.csv")
    df.to_csv(output_csv, index=False)
    print(f"Saved to {output_csv}")

report_name = "missing_vms"

save(_prod, twprod['output_dir'], report_name)
save(_dev, twdev['output_dir'], report_name)
